In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

#print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#Putting all the functions here
#1.1091571486954503

Singles are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
Doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
Total number of excitations = 26
Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.216023859130329


In [2]:
# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):
    qml.BasisState(k, wires=range(qubits))
    #res = qml.commutator(H, w)
    res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k):
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            
    return qml.expval(H)

# Calculation of New state 
#print('Now we are going to calculate the new_state')
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            print('The wires1 are', ash_excitation[i][2:][::-1])
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            

    return qml.state()

In [3]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

print('The Hamiltonian is ', H)
adit = 6
fparams = []
excitations= []
operator_pool = (singles) + (doubles)
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)
        w = qml.fermi.jordan_wigner(i)
        if np.array_equal(k, hf_state):
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))            #Removed the abs() function
        else:
            current_value = abs(2*(commutator_1(H, w, k)))           #Removed the abs() function
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    #ash_excitation.insert(0, excitations)  #Updated this statement
    ash_excitation.append(excitations)
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="adjoint")
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


I changed it both excitations
The Hamiltonian is  -1.1091571486954508 * I(0) + 0.04196325905665612 * Z(0) + 6.339400654100866e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400654100866e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04183466728352534 * Z(2) + 0.01543349646446538 * (Z(0) @ Z(2)) + 6.34455858671988e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.34455858671988e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.004378038401993098 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.004378038401993098 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332477 * Z(4) + 0.01984336499843016 * (Z(0) @ Z(4)) + 0.041576261632175426 * Z(6) + 0.022112211956213074 * (Z(0) @ Z(6)) + 0.04196325905665608 * Z(1) + 0.06277200433822133 * (Z(0) @ Z(1)) + 0.002502564046464182 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.002502564046464182 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.047312950419785164 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.047312950419785164 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.047312950419785164 * (X(0) @

/opt/miniconda3/envs/penny/lib/python3.12/site-packages/autoray/autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


The expectation value of a⁺(4) a(0) is 0.00013218122676719464
The current excitation operator is a⁺(6) a(0)
Print, if this is activated - HF state
The expectation value of a⁺(6) a(0) is 2.9403829191550386e-07
The current excitation operator is a⁺(5) a(1)
Print, if this is activated - HF state
The expectation value of a⁺(5) a(1) is 0.0001321812267658563
The current excitation operator is a⁺(7) a(1)
Print, if this is activated - HF state
The expectation value of a⁺(7) a(1) is 3.377608356901766e-07
The current excitation operator is a⁺(4) a(2)
Print, if this is activated - HF state
The expectation value of a⁺(4) a(2) is 3.5622390345809534e-07
The current excitation operator is a⁺(6) a(2)
Print, if this is activated - HF state
The expectation value of a⁺(6) a(2) is 0.0001331290867369606
The current excitation operator is a⁺(5) a(3)
Print, if this is activated - HF state
The expectation value of a⁺(5) a(3) is 2.8555496333834696e-07
The current excitation operator is a⁺(7) a(3)
Print, if thi

In [ ]:
-1.85544824 - (-1.85546077)
[7, 4, 3, 0][6, 5, 2, 1] 

## Jax interface - optimizer

In [ ]:
import jax
from jax import numpy as jnp
import pennylane as qml
import optax
import jax.numpy as jnp
from pennylane import qchem
from pennylane import numpy as np

jax.config.update("jax_enable_x64", True)


X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 3.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "openfermion")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
qubits = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")



dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    #[qml.PauliX(i) for i in jnp.nonzero(hf_state)[0]] #HF state

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#dev = qml.device("default.qubit", wires=qubits)

#@jax.jit
#@qml.qnode(dev, interface="jax")
#def circuit(param):
#    qml.RX(param, wires=0)
##    qml.CNOT(wires=[0, 1])
#    return qml.expval(H)

#ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]
ash_excitation =  [[3, 7], [0, 4], [1, 2, 4, 7], [0, 1, 4, 5], [0, 3, 5, 6], [2, 3, 6, 7]]

optimizer = optax.adam(learning_rate=0.1)
#optimizer = optax.lbfgs()
params = jnp.zeros(len(ash_excitation), dtype=jnp.float32)
opt_state = optimizer.init(params)
print('The parameters that are going in ', len(params))

dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
#@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            #Exchanged the order of wires 1 and wires 2
            qml.FermionicDoubleExcitation(weight = params[i], wires2=excitations[2:][::-1], wires1=excitations[:2][::-1])
            #Original - qml.FermionicDoubleExcitation(weight = params[i], wires1=excitations[2:][::-1], wires2=excitations[:2][::-1])
        else:
            #print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
    return qml.expval(H)

def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    return energy

def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

gradient_tolerance = 1e-8 
gradient_max_threshold = 1e-8
for i in range(250):
    #cost, grad_circuit = jax.value_and_grad(circuit)(params)
    cost, grad_circuit = jax.value_and_grad(minimize_function)(params)
        # Compute the norm of the gradient
    grad_norm = jnp.linalg.norm(grad_circuit)
    grad_max = jnp.max(jnp.abs(grad_circuit))


    # Print the current gradient norm (optional for debugging)
    print(f"Step {i}, cost: {cost}, grad_norm: {grad_norm}, grad_max: {grad_max}")

    # Check if the gradient norm is below the tolerance
    if grad_norm < gradient_tolerance:
        print(f"Convergence achieved at step {i}. Gradient norm {grad_norm} is below tolerance.")
        break
    if grad_max < gradient_max_threshold:
        print(f"Convergence achieved at step {i}. Maximum gradient {grad_max} is below the threshold.")
        break 
    updates, opt_state = optimizer.update(grad_circuit, opt_state)
    params = optax.apply_updates(params, updates)
    #print(f"step {i}, cost {cost}")
print('Final energy is', cost)